In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from datetime import timedelta

In [2]:
df = pd.read_csv('datasets/raw.csv')
df.head()

,Well,Content,Group,Neg. control correction based on Ratio (485-12 / 520 4),Neg. control correction based on Ratio (485-12 / 520 4).1,Neg. control correction based on Ratio (485-12 / 520 4).2,Neg. control correction based on Ratio (485-12 / 520 4).3,Neg. control correction based on Ratio (485-12 / 520 4).4,Neg. control correction based on Ratio (485-12 / 520 4).5,Neg. control correction based on Ratio (485-12 / 520 4).6,Neg. control correction based on Ratio (485-12 / 520 4).7,Neg. control correction based on Ratio (485-12 / 520 4).8,Neg. control correction based on Ratio (485-12 / 520 4).9,Neg. control correction based on Ratio (485-12 / 520 4).10,Neg. control correction based on Ratio (485-12 / 520 4).11,Neg. control correction based on Ratio (485-12 / 520 4).12,Neg. control correction based on Ratio (485-12 / 520 4).13,Neg. control correction based on Ratio (485-12 / 520 4).14,Neg. control correction based on Ratio (485-12 / 520 4).15,Neg. control correction based on Ratio (485-12 / 520 4).16,Neg. control correction based on Ratio (485-12 / 520 4).17,Neg. control correction based on Ratio (485-12 / 520 4).18,Neg. control correction based on Ratio (485-12 / 520 4).19,Neg. control correction based on Ratio (485-12 / 520 4).20,Neg. control correction based on Ratio (485-12 / 520 4).21,Neg. control correction based on Ratio (485-12 / 520 4).22,Neg. control correction based on Ratio (485-12 / 520 4).23,Neg. control correction based on Ratio (485-12 / 520 4).24,Neg. control correction based on Ratio (485-12 / 520 4).25,Neg. control correction based on Ratio (485-12 / 520 4).26,Neg. control correction based on Ratio (485-12 / 520 4).27,Neg. control correction based on Ratio (485-12 / 520 4).28,Neg. control correction based on Ratio (485-12 / 520 4).29,Neg. control correction based on Ratio (485-12 / 520 4).30,Neg. control correction based on Ratio (485-12 / 520 4).31,Neg. control correction based on Ratio (485-12 / 520 4).32,Neg. control correction based on Ratio (485-12 / 520 4).33,Neg. control correction based on Ratio (485-12 / 520 4).34,Neg. control correction based on Ratio (485-12 / 520 4).35,Neg. control correction based on Ratio (485-12 / 520 4).36,Neg. control correction based on Ratio (485-12 / 520 4).37,Neg. control correction based on Ratio (485-12 / 520 4).38,Neg. control correction based on Ratio (485-12 / 520 4).39,Neg. control correction based on Ratio (485-12 / 520 4).40,Neg. control correction based on Ratio (485-12 / 520 4).41,Neg. control correction based on Ratio (485-12 / 520 4).42,Neg. control correction based on Ratio (485-12 / 520 4).43,Neg. control correction based on Ratio (485-12 / 520 4).44,Neg. control correction based on Ratio (485-12 / 520 4).45,Neg. control correction based on Ratio (485-12 / 520 4).46,Neg. control correction based on Ratio (485-12 / 520 4).47,Neg. control correction based on Ratio (485-12 / 520 4).48,Neg. control correction based on Ratio (485-12 / 520 4).49,Neg. control correction based on Ratio (485-12 / 520 4).50,Neg. control correction based on Ratio (485-12 / 520 4).51,Neg. control correction based on Ratio (485-12 / 520 4).52,Neg. control correction based on Ratio (485-12 / 520 4).53,Neg. control correction based on Ratio (485-12 / 520 4).54,Neg. control correction based on Ratio (485-12 / 520 4).55,Neg. control correction based on Ratio (485-12 / 520 4).56,Neg. control correction based on Ratio (485-12 / 520 4).57,Neg. control correction based on Ratio (485-12 / 520 4).58,Neg. control correction based on Ratio (485-12 / 520 4).59,Neg. control correction based on Ratio (485-12 / 520 4).60,Neg. control correction based on Ratio (485-12 / 520 4).61,Neg. control correction based on Ratio (485-12 / 520 4).62,Neg. control correction based on Ratio (485-12 / 520 4).63,Neg. control correction based on Ratio (485-12 / 520 4).64,Neg. control correction based on Ratio (485-12 / 520 4).65,Neg. control correction based on Ratio (485-12 / 520 4).66,Neg. control correction based on Ratio 

In [3]:
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.dropna(inplace=True)
df.head()

,Well,Content,Group,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h
1,A01,Sample X1,A,-9563.5,4827.62,-2946.32,2144.51,2917.55,1794.39,972.45,724.44,434.34,391.42,212.86,204.1,211.45,310.85,327.35,366.17,446.87,376.73,384.41,431.49,423.22,396.34,413.22,434.08,425.05,459.76,501.51,437.77,489.06,508.58,485.25,523.16,406.34,634.3,473.17,516.08,534.19,676.7,571.85,572.42,544.65,669.69,546.43,578.94,537.18,561.62,523.43,625.26,596.37,635.46,643.03,580.2,713.19,716.23,778.28,902.67,742.11,781.09,789.87,775.07,829.9,795.75,804.28,1149.05,833.41,853.54,885.28,927.34,929.03,939.02,963.05,981.41,1042.26,-0.009,0.017,-0.03,0.029,0.04,0.057,0.088,0.141,0.232,0.337,0.471,0.575,0.74,0.851,0.901,0.943,0.981,1.002,1.034,1.047,1.075,1.082,1.105,1.107,1.103,1.11,1.123,1.123,1.118,1.11,1.125,1.12,1.168,1.112,1.123,1.106,1.101,1.096,1.095,1.094,1.11,1.106,1.099,1.1,1.099,1.101,1.109,1.087,1.098,1.096,1.098,1.109,1.084,1.083,1.026,1.075,1.074,1.077,1.067,1.021,1.053,1.033,1.044,1.039,1.034,1.039,1.024,1.029,1.027,1.019,1.014,1.013,1.004
2,A02,Sample X2,A,-901.91,-612.44,-2021.98,-1082.24,1432.52,908.24,851.45,488.48,375.29,293.11,221.99,203.28,217.27,348.89,318.99,384.01,506.71,429.14,459.42,527.97,507.04,524.05,556.12,597.2,582.37,579.5,600.5,538.32,590.57,631.25,600.38,615.21,635.52,724.82,584.03,615.39,600.26,756.49,683.96,706.57,685.88,842.65,656.97,709.18,673.22,691.76,689.33,664.15,678.11,711.18,688.09,694.01,734.94,773.78,810.51,943.4,813.39,886.01,751.12,858.87,826.18,866.83,857,1160.5,850.36,873.56,922.43,923.53,927.85,939.77,891.52,920.52,956.05,-0.025,0.031,-0.029,0.042,0.046,0.064,0.088,0.133,0.211,0.326,0.422,0.527,0.678,0.805,0.877,0.927,0.966,1.01,1.031,1.05,1.077,1.089,1.098,1.107,1.103,1.113,1.131,1.127,1.132,1.113,1.129,1.131,1.13,1.136,1.137,1.129,1.132,1.115,1.105,1.112,1.112,1.108,1.11,1.112,1.111,1.111,1.114,1.118,1.118,1.114,1.114,1.108,1.116,1.101,1.049,1.107,1.099,1.084,1.085,1.044,1.073,1.036,1.062,1.058,1.056,1.049,1.047,1.04,1.042,1.032,1.034,1.031,1.028
3,A03,Sample X3,A,-15040.52,14858.93,-2139.6,3282.42,2463.19,2427.71,1211.23,858.17,477.14,429.96,235.42,178.07,192.39,211.14,193.53,240.15,352.82,304.27,330.1,392.83,414.42,404.49,410.62,394.97,365.55,411.05,458.15,419.04,512.27,500.38,475.58,500.93,527.05,662.09,530.22,465.53,495.3,624.88,541.26,560.27,499.94,680.23,518.78,561.19,507.34,510.08,478.13,535.85,512.51,550.05,556.38,494.91,577.88,591.5,633.19,782.18,645.82,686.42,615.05,598.45,636.21,647.7,690.17,953.25,649.42,699.29,721.2,706.81,687.56,744.65,734.43,771.71,817.92,-0.005,0.007,-0.021,0.019,0.029,0.036,0.063,0.094,0.167,0.27,0.361,0.479,0.61,0.766,0.855,0.92,0.966,1,1.028,1.047,1.073,1.084,1.123,1.141,1.148,1.137,1.153,1.139,1.134,1.122,1

In [4]:
map_ecf = {'Sample X1': 'e15',
'Sample X2': 'e22',
'Sample X3': 'e32',
'Sample X4': 'e33',
'Sample X5': 'e34',
'Sample X6': 'e41',
'Sample X7': 'e42',
'Sample X8': 'e15',
'Sample X9': 'e22',
'Sample X10': 'e38',
'Sample X11': 'e16',
'Sample X12': 'e33',
'Sample X13': 'e15',
'Sample X14': 'e16',
'Sample X15': 'e17',
'Sample X16': 'e20',
'Sample X17': 'e22',
'Sample X18': 'e26',
'Sample X19': 'e32',
'Sample X20': 'e33',
'Sample X21': 'e34'}
map_int = {'Sample X1': 'SspGyrB',
'Sample X2': 'SspGyrB',
'Sample X3': 'SspGyrB',
'Sample X4': 'SspGyrB',
'Sample X5': 'SspGyrB',
'Sample X6': 'SspGyrB',
'Sample X7': 'SspGyrB',
'Sample X8': 'TerThyXS2',
'Sample X9': 'TerThyXS2',
'Sample X10': 'TerThyXS2',
'Sample X11': 'TerThyXS1',
'Sample X12': 'TerThyXS1',
'Sample X13': 'STPhoRadA',
'Sample X14': 'STPhoRadA',
'Sample X15': 'STPhoRadA',
'Sample X16': 'STPhoRadA',
'Sample X17': 'STPhoRadA',
'Sample X18': 'STPhoRadA',
'Sample X19': 'STPhoRadA',
'Sample X20': 'STPhoRadA',
'Sample X21': 'STPhoRadA'}

In [5]:
df['ECF'] = df['Content'].map(map_ecf)
df['Intein'] = df['Content'].map(map_int)
df.loc[df['Group'].isin(['A', 'E']), 'Ara'] = 0
df.loc[df['Group'].isin(['A', 'E']), 'Cuma'] = 0
df.loc[df['Group'].isin(['B', 'F']), 'Ara'] = 1
df.loc[df['Group'].isin(['B', 'F']), 'Cuma'] = 0
df.loc[df['Group'].isin(['C', 'G']), 'Ara'] = 0
df.loc[df['Group'].isin(['C', 'G']), 'Cuma'] = 1
df.loc[df['Group'].isin(['D', 'H']), 'Ara'] = 1
df.loc[df['Group'].isin(['D', 'H']), 'Cuma'] = 1
df = pd.concat([df[['ECF', 'Intein', 'Ara', 'Cuma']],
                 df.drop(['ECF', 'Intein', 'Ara', 'Cuma', 'Well', 'Content', 'Group'], axis=1)],
                 axis=1)
df.head()

,ECF,Intein,Ara,Cuma,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h
1,e15,SspGyrB,0.0,0.0,-9563.5,4827.62,-2946.32,2144.51,2917.55,1794.39,972.45,724.44,434.34,391.42,212.86,204.1,211.45,310.85,327.35,366.17,446.87,376.73,384.41,431.49,423.22,396.34,413.22,434.08,425.05,459.76,501.51,437.77,489.06,508.58,485.25,523.16,406.34,634.3,473.17,516.08,534.19,676.7,571.85,572.42,544.65,669.69,546.43,578.94,537.18,561.62,523.43,625.26,596.37,635.46,643.03,580.2,713.19,716.23,778.28,902.67,742.11,781.09,789.87,775.07,829.9,795.75,804.28,1149.05,833.41,853.54,885.28,927.34,929.03,939.02,963.05,981.41,1042.26,-0.009,0.017,-0.03,0.029,0.04,0.057,0.088,0.141,0.232,0.337,0.471,0.575,0.74,0.851,0.901,0.943,0.981,1.002,1.034,1.047,1.075,1.082,1.105,1.107,1.103,1.11,1.123,1.123,1.118,1.11,1.125,1.12,1.168,1.112,1.123,1.106,1.101,1.096,1.095,1.094,1.11,1.106,1.099,1.1,1.099,1.101,1.109,1.087,1.098,1.096,1.098,1.109,1.084,1.083,1.026,1.075,1.074,1.077,1.067,1.021,1.053,1.033,1.044,1.039,1.034,1.039,1.024,1.029,1.027,1.019,1.014,1.013,1.004
2,e22,SspGyrB,0.0,0.0,-901.91,-612.44,-2021.98,-1082.24,1432.52,908.24,851.45,488.48,375.29,293.11,221.99,203.28,217.27,348.89,318.99,384.01,506.71,429.14,459.42,527.97,507.04,524.05,556.12,597.2,582.37,579.5,600.5,538.32,590.57,631.25,600.38,615.21,635.52,724.82,584.03,615.39,600.26,756.49,683.96,706.57,685.88,842.65,656.97,709.18,673.22,691.76,689.33,664.15,678.11,711.18,688.09,694.01,734.94,773.78,810.51,943.4,813.39,886.01,751.12,858.87,826.18,866.83,857,1160.5,850.36,873.56,922.43,923.53,927.85,939.77,891.52,920.52,956.05,-0.025,0.031,-0.029,0.042,0.046,0.064,0.088,0.133,0.211,0.326,0.422,0.527,0.678,0.805,0.877,0.927,0.966,1.01,1.031,1.05,1.077,1.089,1.098,1.107,1.103,1.113,1.131,1.127,1.132,1.113,1.129,1.131,1.13,1.136,1.137,1.129,1.132,1.115,1.105,1.112,1.112,1.108,1.11,1.112,1.111,1.111,1.114,1.118,1.118,1.114,1.114,1.108,1.116,1.101,1.049,1.107,1.099,1.084,1.085,1.044,1.073,1.036,1.062,1.058,1.056,1.049,1.047,1.04,1.042,1.032,1.034,1.031,1.028
3,e32,SspGyrB,0.0,0.0,-15040.52,14858.93,-2139.6,3282.42,2463.19,2427.71,1211.23,858.17,477.14,429.96,235.42,178.07,192.39,211.14,193.53,240.15,352.82,304.27,330.1,392.83,414.42,404.49,410.62,394.97,365.55,411.05,458.15,419.04,512.27,500.38,475.58,500.93,527.05,662.09,530.22,465.53,495.3,624.88,541.26,560.27,499.94,680.23,518.78,561.19,507.34,510.08,478.13,535.85,512.51,550.05,556.38,494.91,577.88,591.5,633.19,782.18,645.82,686.42,615.05,598.45,636.21,647.7,690.17,953.25,649.42,699.29,721.2,706.81,687.56,744.65,734.43,771.71,817.92,-0.005,0.007,-0.021,0.019,0.029,0.036,0.063,0.094,0.167,0.27,0.361,0.479,0.61,0.766,0.855,0.92,0.966,1,1.028,1.047,1.073,1.084,1.123,1.141,1.148,1.137,1.153,1.139,

In [6]:
cols = list(range(0,77))
fluo = df.iloc[:, cols]
#fluo.to_csv('datasets/fluoOD-all.csv', index=False)
fluo.head()

,ECF,Intein,Ara,Cuma,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h
1,e15,SspGyrB,0.0,0.0,-9563.5,4827.62,-2946.32,2144.51,2917.55,1794.39,972.45,724.44,434.34,391.42,212.86,204.1,211.45,310.85,327.35,366.17,446.87,376.73,384.41,431.49,423.22,396.34,413.22,434.08,425.05,459.76,501.51,437.77,489.06,508.58,485.25,523.16,406.34,634.3,473.17,516.08,534.19,676.7,571.85,572.42,544.65,669.69,546.43,578.94,537.18,561.62,523.43,625.26,596.37,635.46,643.03,580.2,713.19,716.23,778.28,902.67,742.11,781.09,789.87,775.07,829.9,795.75,804.28,1149.05,833.41,853.54,885.28,927.34,929.03,939.02,963.05,981.41,1042.26
2,e22,SspGyrB,0.0,0.0,-901.91,-612.44,-2021.98,-1082.24,1432.52,908.24,851.45,488.48,375.29,293.11,221.99,203.28,217.27,348.89,318.99,384.01,506.71,429.14,459.42,527.97,507.04,524.05,556.12,597.2,582.37,579.5,600.5,538.32,590.57,631.25,600.38,615.21,635.52,724.82,584.03,615.39,600.26,756.49,683.96,706.57,685.88,842.65,656.97,709.18,673.22,691.76,689.33,664.15,678.11,711.18,688.09,694.01,734.94,773.78,810.51,943.4,813.39,886.01,751.12,858.87,826.18,866.83,857,1160.5,850.36,873.56,922.43,923.53,927.85,939.77,891.52,920.52,956.05
3,e32,SspGyrB,0.0,0.0,-15040.52,14858.93,-2139.6,3282.42,2463.19,2427.71,1211.23,858.17,477.14,429.96,235.42,178.07,192.39,211.14,193.53,240.15,352.82,304.27,330.1,392.83,414.42,404.49,410.62,394.97,365.55,411.05,458.15,419.04,512.27,500.38,475.58,500.93,527.05,662.09,530.22,465.53,495.3,624.88,541.26,560.27,499.94,680.23,518.78,561.19,507.34,510.08,478.13,535.85,512.51,550.05,556.38,494.91,577.88,591.5,633.19,782.18,645.82,686.42,615.05,598.45,636.21,647.7,690.17,953.25,649.42,699.29,721.2,706.81,687.56,744.65,734.43,771.71,817.92
4,e33,SspGyrB,0.0,0.0,210.74,19396.44,-210.92,3274.99,2949.69,2078.02,1262.65,878.48,330.1,435.19,200.13,153.38,126.9,170.07,176.98,189.78,293.51,207.99,241.15,296.15,267.09,264.18,251.39,291.8,250.24,268.55,290.86,261.69,315.38,307.25,307.76,334.17,343.79,462.01,311.97,321.87,330.86,480.58,395.09,356.38,318.49,481.5,399.93,416.66,350.64,356.73,392.5,371.12,376.89,338.16,412.53,352.27,440.84,439.07,479.28,653.87,482.63,550.35,388.36,424.14,428.43,499.13,444.73,760.15,437.05,468.08,553.76,562.9,525.11,565.74,494.02,525.51,557.76
5,e34,SspGyrB,0.0,0.0,-687.59,16509.84,-801.39,3383.05,2385.7,2822.14,1158.82,826.29,486.39,394.26,329.62,254.74,323.58,366.4,420.05,479.56,579.66,497.77,533.52,580.82,566.78,536.78,553.2,582.92,574.13,611.11,620.93,588.11,595.45,626.12,594.33,650.61,654.42,763.26,653.22,646.57,666.43,819.07,694.18,690.94,695.13,869.33,735,725.07,718.75,703.43,752.54,791.17,805.29,784.55,798.68,786.17,830.54,855.31,896.48,1060.14,887.05,969.17,840.08,891.86,907.24,971.74,949.36,1250.4,937.08,982.35,1010.43,1032.36,1031.46,1024.81,1027.47,1009.38,1049.27


In [7]:
cols = list(range(0,4)) + list(range(77,150))
od = df.iloc[:, cols]
#od.to_csv('datasets/OD-all.csv', index=False)
od.head()

,ECF,Intein,Ara,Cuma,0 h,0 h 20 min,0 h 40 min,1 h,1 h 20 min,1 h 40 min,2 h,2 h 20 min,2 h 40 min,3 h,3 h 20 min,3 h 40 min,4 h,4 h 20 min,4 h 40 min,5 h,5 h 20 min,5 h 40 min,6 h,6 h 20 min,6 h 40 min,7 h,7 h 20 min,7 h 40 min,8 h,8 h 20 min,8 h 40 min,9 h,9 h 20 min,9 h 40 min,10 h,10 h 20 min,10 h 40 min,11 h,11 h 20 min,11 h 40 min,12 h,12 h 20 min,12 h 40 min,13 h,13 h 20 min,13 h 40 min,14 h,14 h 20 min,14 h 40 min,15 h,15 h 20 min,15 h 40 min,16 h,16 h 20 min,16 h 40 min,17 h,17 h 20 min,17 h 40 min,18 h,18 h 20 min,18 h 40 min,19 h,19 h 20 min,19 h 40 min,20 h,20 h 20 min,20 h 40 min,21 h,21 h 20 min,21 h 40 min,22 h,22 h 20 min,22 h 40 min,23 h,23 h 20 min,23 h 40 min,24 h
1,e15,SspGyrB,0.0,0.0,-0.009,0.017,-0.03,0.029,0.04,0.057,0.088,0.141,0.232,0.337,0.471,0.575,0.74,0.851,0.901,0.943,0.981,1.002,1.034,1.047,1.075,1.082,1.105,1.107,1.103,1.11,1.123,1.123,1.118,1.11,1.125,1.12,1.168,1.112,1.123,1.106,1.101,1.096,1.095,1.094,1.11,1.106,1.099,1.1,1.099,1.101,1.109,1.087,1.098,1.096,1.098,1.109,1.084,1.083,1.026,1.075,1.074,1.077,1.067,1.021,1.053,1.033,1.044,1.039,1.034,1.039,1.024,1.029,1.027,1.019,1.014,1.013,1.004
2,e22,SspGyrB,0.0,0.0,-0.025,0.031,-0.029,0.042,0.046,0.064,0.088,0.133,0.211,0.326,0.422,0.527,0.678,0.805,0.877,0.927,0.966,1.01,1.031,1.05,1.077,1.089,1.098,1.107,1.103,1.113,1.131,1.127,1.132,1.113,1.129,1.131,1.13,1.136,1.137,1.129,1.132,1.115,1.105,1.112,1.112,1.108,1.11,1.112,1.111,1.111,1.114,1.118,1.118,1.114,1.114,1.108,1.116,1.101,1.049,1.107,1.099,1.084,1.085,1.044,1.073,1.036,1.062,1.058,1.056,1.049,1.047,1.04,1.042,1.032,1.034,1.031,1.028
3,e32,SspGyrB,0.0,0.0,-0.005,0.007,-0.021,0.019,0.029,0.036,0.063,0.094,0.167,0.27,0.361,0.479,0.61,0.766,0.855,0.92,0.966,1,1.028,1.047,1.073,1.084,1.123,1.141,1.148,1.137,1.153,1.139,1.134,1.122,1.143,1.143,1.138,1.12,1.122,1.121,1.128,1.129,1.121,1.123,1.129,1.12,1.122,1.123,1.124,1.129,1.142,1.13,1.14,1.138,1.128,1.129,1.133,1.125,1.074,1.12,1.117,1.111,1.113,1.08,1.1,1.068,1.082,1.078,1.074,1.072,1.06,1.072,1.075,1.058,1.044,1.037,1.03
4,e33,SspGyrB,0.0,0.0,-0.04,0.005,-0.036,0.016,0.023,0.035,0.061,0.096,0.173,0.28,0.377,0.489,0.631,0.787,0.856,0.92,0.967,1.007,1.028,1.041,1.083,1.091,1.128,1.135,1.145,1.145,1.162,1.151,1.16,1.14,1.16,1.154,1.148,1.146,1.148,1.136,1.139,1.133,1.126,1.131,1.144,1.131,1.119,1.127,1.121,1.128,1.131,1.124,1.13,1.141,1.119,1.117,1.122,1.114,1.06,1.105,1.104,1.092,1.118,1.073,1.107,1.069,1.09,1.083,1.08,1.078,1.067,1.066,1.066,1.057,1.055,1.051,1.041
5,e34,SspGyrB,0.0,0.0,-0.032,0.005,-0.037,0.015,0.023,0.034,0.062,0.095,0.174,0.286,0.382,0.492,0.63,0.779,0.84,0.887,0.939,0.976,1.001,1.023,1.054,1.071,1.095,1.102,1.108,1.109,1.121,1.112,1.117,1.107,1.128,1.119,1.123,1.122,1.117,1.109,1.119,1.099,1.099,1.101,1.101,1.099,1.092,1.106,1.092,1.093,1.097,1.091,1.092,1.094,1.088,1.094,1.092,1.083,1.029,1.08,1.074,1.067,1.074,1.032,1.057,1.024,1.046,1.038,1.033,1.033,1.026,1.025,1.02,1.017,1.008,1.004,0.993


In [18]:
def parse_minutes(x):
    spl = x.split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

fluo11 = fluo[(fluo['Ara']==1) & (fluo['Cuma']==0)]
fluo11['index'] = fluo11['ECF'] + fluo11['Intein']
fluo11.set_index('index', inplace=True)
fluo11.drop(['ECF', 'Intein', 'Ara', 'Cuma'], axis=1, inplace=True)
fluo11 = fluo11.transpose().reset_index()

fluo11['time'] = fluo11[0].apply(parse_minutes)
fluo11 = fluo11.set_index('time')
fluo11 = fluo11.drop(0, axis=1)
fluo11

<ipython-input-18-f99292f17b99>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluo11['index'] = fluo11['ECF'] + fluo11['Intein']
/Users/mardian/Documents/Edinburgh/Notebook/env/lib/python3.9/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


index,e15SspGyrB,e22SspGyrB,e32SspGyrB,e33SspGyrB,e34SspGyrB,e41SspGyrB,e42SspGyrB,e15TerThyXS2,e22TerThyXS2,e38TerThyXS2,e16TerThyXS1,e33TerThyXS1,e15STPhoRadA,e16STPhoRadA,e17STPhoRadA,e20STPhoRadA,e22STPhoRadA,e26STPhoRadA,e32STPhoRadA,e33STPhoRadA,e34STPhoRadA
time,,,,,,,,,,,,,,,,,,,,,
0,-1872.6,-1335,-957.62,-2584.98,-1494.56,-706.64,-1338.21,-2590.15,-2521.38,5814.66,-1815.9,-2574.43,-1945.49,-2051.74,-2609.42,-1242.81,-2254.83,-904.92,-888.57,-2454.93,-1512.01
20,-111472.87,-101857.14,-115319.15,-176923.08,-150312.5,-19090.91,-83076.92,-147027.03,-97555.56,-158750,-114228.19,-116470.59,-114340.66,-101400,-75000,-96779.66,-63888.89,-86296.3,-106716.42,n.a.,-144242.42
40,-2898.82,-52.68,-2150.28,-1037.58,-1473.96,-187.51,-1932.2,-746.74,-132.47,-21.74,-2805.49,-4298.33,-2503.91,-2818.06,-1241.48,-1972.72,-3265.15,-2374.09,-3187.92,-2547.24,-582.61
60,3138.83,5731.4,1840.2,7547.82,14246.72,5528.77,5009.96,11042.55,7404.82,8267.51,1988.54,605.83,1624.17,2523.43,3845.04,4889.5,4310.61,4421.67,1152.18,1796.47,-611.87
80,-1173.22,1253.5,-74.98,585.91,644.16,561.43,1316.4,1878.79,2130.57,-313.09,-1573.18,-2189.54,-1764.31,-139.91,-913.65,427.9,-18.79,-2622.53,-1646.27,-1786.1,-47.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,1063.48,1455.19,1065.08,910.51,1286.2,1183.81,878.41,711.65,1111.22,1297.43,581.73,512.44,778.9,850.96,1143.52,1044.54,1080.5,904.53,927.14,286.01,1155.47
1380,1378.84,1666.31,1333.35,1090.97,1565.7,1423.57,1121.81,948.96,1396.85,1522.11,861.43,750.52,1089.75,1099.28,1385.62,1647.48,1365.89,1157.56,1208.13,752.26,1401.11
1400,1196,1538.92,1149.44,922.65,1315.01,1227.06,910.87,776.23,1196.16,1290.01,712.33,555.95,864.51,907.24,1221.89,1464.1,1183.05,925.77,1020.98,532.29,1219.16


In [19]:
od11 = od[(od['Ara']==1) & (od['Cuma']==0)]
od11['index'] = od11['ECF'] + od11['Intein']
od11.set_index('index', inplace=True)
od11.drop(['ECF', 'Intein', 'Ara', 'Cuma'], axis=1, inplace=True)
od11 = od11.transpose().reset_index()

od11['time'] = od11[0].apply(parse_minutes)
od11 = od11.set_index('time')
od11 = od11.drop(0, axis=1)
od11

<ipython-input-19-22853c8867df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  od11['index'] = od11['ECF'] + od11['Intein']
/Users/mardian/Documents/Edinburgh/Notebook/env/lib/python3.9/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


index,e15SspGyrB,e22SspGyrB,e32SspGyrB,e33SspGyrB,e34SspGyrB,e41SspGyrB,e42SspGyrB,e15TerThyXS2,e22TerThyXS2,e38TerThyXS2,e16TerThyXS1,e33TerThyXS1,e15STPhoRadA,e16STPhoRadA,e17STPhoRadA,e20STPhoRadA,e22STPhoRadA,e26STPhoRadA,e32STPhoRadA,e33STPhoRadA,e34STPhoRadA
time,,,,,,,,,,,,,,,,,,,,,
0,0.031,0.039,0.042,0.046,0.036,0.034,0.039,0.063,0.067,0.015,0.031,0.053,0.014,0.031,0.038,0.052,0.05,0.029,0.031,0.041,0.035
20,0.013,0.007,0.023,-0.001,-0.003,0.001,0.003,-0.004,0.005,-0.002,0.015,0.031,0.018,0.005,0.002,0.006,0.002,0.003,0.007,0,-0.003
40,0.027,0.021,0.031,0.018,0.015,0.018,0.021,0.018,0.015,0.013,0.019,0.035,0.018,0.02,0.017,0.024,0.021,0.016,0.022,0.017,0.013
60,0.024,0.016,0.034,0.009,0.006,0.013,0.013,0.007,0.01,0.009,0.022,0.038,0.03,0.022,0.015,0.018,0.015,0.014,0.02,0.013,0.091
80,0.037,0.026,0.037,0.018,0.017,0.024,0.023,0.016,0.02,0.027,0.032,0.053,0.04,0.027,0.028,0.029,0.027,0.061,0.036,0.025,0.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,0.967,0.995,0.985,0.932,0.954,0.963,0.968,0.98,0.984,0.983,1.038,1.054,1.034,1.036,1.022,1.052,1.053,1.03,0.994,1.127,0.992
1380,0.953,0.996,0.987,0.944,0.95,0.958,0.968,0.974,0.976,0.979,1.028,1.051,1.032,1.033,1.019,0.971,1.047,1.028,0.986,1.062,0.985
1400,0.945,0.985,0.974,0.933,0.943,0.942,0.955,0.965,0.96,0.973,1.005,1.045,1.027,1.025,1.01,0.958,1.033,1.019,0.974,1.053,0.977


In [26]:
for col in fluo11.columns:
    try:
        fluo11[col] = fluo11[col].astype(float)
    except:
        print(col)
        continue

In [27]:
for col in od11.columns:
    od11[col] = od11[col].astype(float)

In [28]:
fluo11.to_csv('datasets/fluo-10.csv')
od11.to_csv('datasets/od-10.csv')
(fluo11 * od11).to_csv('datasets/bulk-fluo-10.csv')